In [0]:
!pip install transformers

     |████████████████████████████████| 317kB 6.3MB/s 
     |████████████████████████████████| 645kB 50.6MB/s 
     |████████████████████████████████| 860kB 41.6MB/s 
     |████████████████████████████████| 1.0MB 43.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=d90f18470811523048d757255858b2c115686b7fef87a47ba05e006e63ddfc25
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


## BERT Fine-tuning on CMRC2018

使用 [Transformers](https://github.com/huggingface/transformers) 封装的 BERT fine-tunning on CMRC2018，参考 [Fine-tuning on SQuAD](https://huggingface.co/transformers/examples.html#fine-tuning-on-squad)。

CMRC2018 数据集 SQuAD 格式:

```json
{'version' : 'v1.0', 
 'data' : [  // 2403 个段落列表
           {'paragraphs' : [{'id':'', 
                             'context':'', 
                             'qas':[{'question':'', 
                                     'id':'', 
                                     'answers':[{'text': '', 
                                                 'answer_start': 41}]
                                    }, 
                                    {'question':'', 'id':'', 'answers':[{}]},
                                    ...
                                   ]
                           }], 
            'id' : 'TRAIN_186', 
            'title': '范廷颂' 
           },
           {'paragraphs' : [{'id', 'context', 'qas'}], 'id' : '', 'title': ''}, 
           ...
          ] }
```




1.   需要重新组织token，处理年代问题！
2.   列表项



In [0]:
import os
import json
import math
import logging
import collections
from io import open

from tqdm import tqdm, trange

In [0]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                TensorDataset)

In [0]:
from transformers import (WEIGHTS_NAME, BertConfig,
                          BertForQuestionAnswering, BertTokenizer)
from transformers import AdamW, WarmupLinearSchedule
from transformers.tokenization_bert import BasicTokenizer, whitespace_tokenize

In [0]:
logger = logging.getLogger(__name__)

### 设置参数

In [0]:
class Arguments(object):
    do_train = True  # Whether to run training.
    # 模型参数
    model_type = 'bert'
    config_name = ''  # Pretrained config name or path if not the same as model_name
    model_name_or_path = 'bert-base-chinese'  # Path to pre-trained model or shortcut name
    tokenizer_name = ''  # Pretrained tokenizer name or path if not the same as model_name
    do_lower_case = True  # Set this flag if you are using an uncased model.
    
    # 数据参数
    train_file = './data/cmrc2018_train.json'  # 训练集文件路径
    dev_file = './data/cmrc2018_dev.json'  # 测试集文件路径
    version_2_with_negative = False  # If true, the SQuAD examples contain some that do not have an answer.
    # The maximum total input sequence length after WordPiece tokenization. Sequences. 
    # longer than this will be truncated, and sequences shorter than this will be padded."
    max_seq_length = 512
    doc_stride = 128  # When splitting up a long document into chunks, how much stride to take between chunks.
    max_query_length = 64  # The maximum number of tokens for the question. Questions longer than this will be truncated to this length.
    
    overwrite_cache = True  # Overwrite the cached training and evaluation sets
    
    # 训练参数
    learning_rate = 5e-5  # The initial learning rate for Adam.
    adam_epsilon = 1e-8  # Epsilon for Adam optimizer.
    train_batch_size = 8  # batch size train
    eval_batch_size = 8  # batch size eval
    gradient_accumulation_steps = 1  # Number of updates steps to accumulate before performing a backward/update pass.
    max_steps = -1  # If > 0: set total number of training steps to perform. Override num_train_epochs.
    num_train_epochs = 3  # Total number of training epochs to perform.
    max_grad_norm = 1.0  # Max gradient norm.
    weight_decay = 0.0  # Weight deay if we apply some.
    warmup_steps = 0.0  # Linear warmup over warmup_steps.
    logging_steps = 50  # Log every X updates steps.
    evaluate_during_training = False  # Rul evaluation during training at each logging step.
    save_steps = 50  # Save checkpoint every X updates steps.
    
    output_dir = '/tmp/debug_squad/'  # The output directory where the model checkpoints and predictions will be written.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
args = Arguments()

In [0]:
# 设置模型
MODEL_CLASSES = {
    'bert': (BertConfig, BertForQuestionAnswering, BertTokenizer),
#     'xlnet': (XLNetConfig, XLNetForQuestionAnswering, XLNetTokenizer),
#     'xlm': (XLMConfig, XLMForQuestionAnswering, XLMTokenizer),
#     'distilbert': (DistilBertConfig, DistilBertForQuestionAnswering, DistilBertTokenizer)
}

### 加载数据

#### 数据预处理

构建 SQuAD 样本类

In [0]:
class SquadExample(object):
    """SQuAD样本
    A single training/test example for the Squad dataset.
    For examples without an answer, the start and end position are -1.
    """

    def __init__(self,
                 qas_id,
                 question_text,
                 doc_tokens,
                 orig_answer_text=None,
                 start_position=None,
                 end_position=None,
                 is_impossible=None):
        self.qas_id = qas_id  # id
        self.question_text = question_text  # 问题
        self.doc_tokens = doc_tokens  # 段落
        self.orig_answer_text = orig_answer_text  # 答案
        self.start_position = start_position  # 答案起始位置
        self.end_position = end_position  # 答案终止位置
        self.is_impossible = is_impossible  # 是否一定有答案

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "qas_id: %s" % (self.qas_id)
        s += ", question_text: %s" % (
            self.question_text)
        s += ", doc_tokens: [%s]" % (" ".join(self.doc_tokens))
        if self.start_position:
            s += ", start_position: %d" % (self.start_position)
        if self.end_position:
            s += ", end_position: %d" % (self.end_position)
        if self.is_impossible:
            s += ", is_impossible: %r" % (self.is_impossible)
        return s


读取文件，构建样本列表

```json
[{'id': 'TRAIN_186',
  'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；...',
  'qas': [{'question': '范廷颂是什么时候被任为主教的？',
           'id': 'TRAIN_186_QUERY_0',
           'answers': [{'text': '1963年', 'answer_start': 30}]},
          {'question': '1990年，范廷颂担任什么职务？',
           'id': 'TRAIN_186_QUERY_1',
           'answers': [{'text': '1990年被擢升为天主教河内总教区宗座署理', 'answer_start': 41}]},
          ...
         ]
 }]
```

In [0]:
def read_squad_examples(input_file, is_training, version_2_with_negative):
    """Read a SQuAD json file into a list of SquadExample. 读取 SQuAD json文件，返回一个 SquadExample 列表 list
    input_file：json 文件路径
    is_training：是否用于训练
    version_2_with_negative：版本2中的一些问题没有答案。If true, the SQuAD examples contain some that do not have an answer.
    """
    # 读取文件
    with open(input_file, "r", encoding='utf-8') as reader:
        input_data = json.load(reader)["data"]

    def is_whitespace(c):  # 注意不需要空格
        if c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False

    examples = []
    for pn,entry in enumerate(input_data):
#         print(pn)
        for paragraph in entry["paragraphs"]:
            # 去除段落中字间的空白字符
            paragraph_text = paragraph["context"]
            doc_tokens = []
            word_offset = []
#             prev_is_whitespace = True
            for w in paragraph_text:
                if is_whitespace(w):
                    continue
#                     prev_is_whitespace = True
                else:
                    doc_tokens.append(w)
#                     if prev_is_whitespace:
#                         doc_tokens.append(c)
#                     else:
#                         doc_tokens[-1] += c
#                     prev_is_whitespace = False
                word_offset.append(len(doc_tokens) - 1)
#             print(len(doc_tokens), len(word_offset))
#             return None
            for qa in paragraph["qas"]:
                qas_id = qa["id"]
                question_text = qa["question"]
                start_position = None
                end_position = None
                orig_answer_text = None
                is_impossible = False
                if is_training:
                    if version_2_with_negative:
                        is_impossible = qa["is_impossible"]
                    if (len(qa["answers"]) != 1) and (not is_impossible):
                        raise ValueError(
                            "For training, each question should have exactly 1 answer.")
                    if not is_impossible:
                        answer = qa["answers"][0]
                        orig_answer_text = answer["text"]
                        answer_offset = answer["answer_start"]
                        answer_length = len(orig_answer_text)
                        start_position = word_offset[answer_offset]
                        # 防止位置标记出错，导致end position 超出列表长度
                        if answer_offset + answer_length - 1 >= len(word_offset):
                            end_position = word_offset[-1]
                        else:
                            end_position = word_offset[answer_offset + answer_length - 1]  # 出问题了！
                        # Only add answers where the text can be exactly recovered from the document.
                        # If this CAN'T happen it's likely due to weird Unicode stuff so we will just skip the example.
                        # Note that this means for training mode, every example is NOT guaranteed to be preserved.
                        actual_text = "".join(doc_tokens[start_position:(end_position + 1)])
                        # Runs basic whitespace cleaning and splitting on a piece of text.
                        # cleaned_answer_text = "".join(whitespace_tokenize(orig_answer_text))
                        cleaned_answer_text = orig_answer_text
                        if actual_text.find(cleaned_answer_text) == -1:
                            # 如果标记出错，尝试重新标记
                            new_answer_offset = paragraph_text.find(orig_answer_text)
                            if new_answer_offset > 0:
                                start_position = word_offset[new_answer_offset]
                            else:
                                logger.warning("Could not find answer: '%s' vs. '%s'",actual_text, cleaned_answer_text)
                                continue
                    else:
                        start_position = -1
                        end_position = -1
                        orig_answer_text = ""

                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    doc_tokens=doc_tokens,
                    orig_answer_text=orig_answer_text,
                    start_position=start_position,
                    end_position=end_position,
                    is_impossible=is_impossible)
                examples.append(example)
    
    return examples


#### Features

In [0]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 doc_span_index,
                 tokens,
                 token_to_orig_map,
                 token_is_max_context,
                 input_ids,
                 input_mask,
                 segment_ids,
                 cls_index,
                 p_mask,
                 paragraph_len,
                 start_position=None,
                 end_position=None,
                 is_impossible=None):
        self.unique_id = unique_id
        self.example_index = example_index
        self.doc_span_index = doc_span_index
        self.tokens = tokens
        self.token_to_orig_map = token_to_orig_map
        self.token_is_max_context = token_is_max_context
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.cls_index = cls_index
        self.p_mask = p_mask
        self.paragraph_len = paragraph_len
        self.start_position = start_position
        self.end_position = end_position
        self.is_impossible = is_impossible


In [0]:
def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer,
                         orig_answer_text):
    """Returns tokenized answer spans that better match the annotated answer. 处理答案，让答案与原文更匹配"""

    # The SQuAD annotations are character based. We first project them to whitespace-tokenized words.
    # But then after WordPiece tokenization, we can often find a "better match". For example:
    #
    #   Question: What year was John Smith born?
    #   Context: The leader was John Smith (1895-1943).
    #   Answer: 1895
    #
    # The original whitespace-tokenized answer will be "(1895-1943).". However
    # after tokenization, our tokens will be "( 1895 - 1943 ) .". So we can match
    # the exact answer, 1895.
    #
    # However, this is not always possible. Consider the following:
    #
    #   Question: What country is the top exporter of electornics?
    #   Context: The Japanese electronics industry is the lagest in the world.
    #   Answer: Japan
    #
    # In this case, the annotator chose "Japan" as a character sub-span of the word "Japanese".
    # Since our WordPiece tokenizer does not split "Japanese", we just use "Japanese" as the annotation.
    # This is fairly rare in SQuAD, but does happen.
    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))
    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start:(new_end + 1)])
            if text_span == tok_answer_text:
                return (new_start, new_end)

    return (input_start, input_end)


In [0]:
def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token. 选取一个答案在文本中间的段落"""

    # Because of the sliding window approach taken to scoring documents, a single token can appear in multiple documents. E.g.
    #  Doc: the man went to the store and bought a gallon of milk
    #  Span A: the man went to the
    #  Span B: to the store and bought
    #  Span C: and bought a gallon of
    #  ...
    #
    # Now the word 'bought' will have two scores from spans B and C. We only
    # want to consider the score with "maximum context", which we define as
    # the *minimum* of its left and right context (the *sum* of left and
    # right context will always be the same, of course).
    #
    # In the example the maximum context for 'bought' would be span C since
    # it has 1 left context and 3 right context, while span B has 4 left context
    # and 0 right context.
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index


将 examples 转换为 feature 类。

In [0]:
def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 doc_stride, max_query_length, is_training,
                                 cls_token_at_end=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=0, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """Loads a data file into a list of `InputBatch`s. 将样本转换为 feature """

    unique_id = 1000000000
    # cnt_pos, cnt_neg = 0, 0
    # max_N, max_M = 1024, 1024
    # f = np.zeros((max_N, max_M), dtype=np.float32)

    features = []
    for (example_index, example) in enumerate(examples):

        # if example_index % 100 == 0:
        #     logger.info('Converting %s/%s pos %s neg %s', example_index, len(examples), cnt_pos, cnt_neg)

        query_tokens = tokenizer.tokenize(example.question_text)

        if len(query_tokens) > max_query_length:
            query_tokens = query_tokens[0:max_query_length]
        # 由于使用 word pisces tokenize
        tok_to_orig_index = []  # tokenize 后当前 token 在 doc text 中的原始索引  [1, 2, 2, 3, 3, 4, ...]
        orig_to_tok_index = []  # 原始 token 在 tokenize 后的索引节点 [0, 1, 3, 5, ...]
        all_doc_tokens = []  # tokenize 后的token
        for (i, token) in enumerate(example.doc_tokens):
            orig_to_tok_index.append(len(all_doc_tokens))
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                tok_to_orig_index.append(i)
                all_doc_tokens.append(sub_token)

        tok_start_position = None
        tok_end_position = None
        if is_training and example.is_impossible:
            tok_start_position = -1
            tok_end_position = -1
        if is_training and not example.is_impossible:
            tok_start_position = orig_to_tok_index[example.start_position]
            if example.end_position < len(example.doc_tokens) - 1:
                tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
            else:
                tok_end_position = len(all_doc_tokens) - 1
            (tok_start_position, tok_end_position) = _improve_answer_span(
                all_doc_tokens, tok_start_position, tok_end_position, tokenizer,
                example.orig_answer_text)

        # The -3 accounts for [CLS], [SEP] and [SEP]
        max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

        # We can have documents that are longer than the maximum sequence length.  处理过长文本？
        # To deal with this we do a sliding window approach, where we take chunks
        # of the up to our max length with a stride of `doc_stride`.
        _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name  命名 tuple
            "DocSpan", ["start", "length"])
        doc_spans = []
        start_offset = 0
        while start_offset < len(all_doc_tokens):
            length = len(all_doc_tokens) - start_offset
            if length > max_tokens_for_doc:
                length = max_tokens_for_doc
            doc_spans.append(_DocSpan(start=start_offset, length=length))
            if start_offset + length == len(all_doc_tokens):
                break
            start_offset += min(length, doc_stride)

        for (doc_span_index, doc_span) in enumerate(doc_spans):
            tokens = []  # 用于保存 token 的列表 [CLS, query token, SEP, paragraph, SEP]
            token_to_orig_map = {}  # 用于保存 token 与 doc word pisces token 索引映射
            token_is_max_context = {}  # 是否是最合适的段落
            segment_ids = []  # segment id [0, 0, 0, 1, 1, 1] 用于标记 segment
            # p_mask 用于遮盖不用于答案的部分
            # p_mask: mask with 1 for token than cannot be in the answer (0 for token which can be in an answer)
            # Original TF implem also keep the classification token (set to 0) (not sure why...)
            p_mask = []

            # CLS token at the beginning
            if not cls_token_at_end:
                tokens.append(cls_token)
                segment_ids.append(cls_token_segment_id)
                p_mask.append(0)
                cls_index = 0

            # Query  问题序列
            for token in query_tokens:
                tokens.append(token)
                segment_ids.append(sequence_a_segment_id)
                p_mask.append(1)

            # SEP token
            tokens.append(sep_token)
            segment_ids.append(sequence_a_segment_id)
            p_mask.append(1)

            # Paragraph
            for i in range(doc_span.length):
                split_token_index = doc_span.start + i
                token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

                is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                       split_token_index)
                token_is_max_context[len(tokens)] = is_max_context
                tokens.append(all_doc_tokens[split_token_index])
                segment_ids.append(sequence_b_segment_id)
                p_mask.append(0)
            paragraph_len = doc_span.length

            # SEP token
            tokens.append(sep_token)
            segment_ids.append(sequence_b_segment_id)
            p_mask.append(1)

            # CLS token at the end
            if cls_token_at_end:
                tokens.append(cls_token)
                segment_ids.append(cls_token_segment_id)
                p_mask.append(0)
                cls_index = len(tokens) - 1  # Index of classification token

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            # Zero-pad up to the sequence length.  模型输入的主要数据 4 个
            while len(input_ids) < max_seq_length:
                input_ids.append(pad_token)
                input_mask.append(0 if mask_padding_with_zero else 1)
                segment_ids.append(pad_token_segment_id)
                p_mask.append(1)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            span_is_impossible = example.is_impossible
            start_position = None
            end_position = None
            if is_training and not span_is_impossible:
                # For training, if our document chunk does not contain an annotation
                # we throw it out, since there is nothing to predict.
                doc_start = doc_span.start
                doc_end = doc_span.start + doc_span.length - 1
                out_of_span = False
                if not (tok_start_position >= doc_start and
                        tok_end_position <= doc_end):
                    out_of_span = True
                if out_of_span:
                    start_position = 0
                    end_position = 0
                    span_is_impossible = True
                else:
                    doc_offset = len(query_tokens) + 2
                    start_position = tok_start_position - doc_start + doc_offset
                    end_position = tok_end_position - doc_start + doc_offset

            if is_training and span_is_impossible:
                start_position = cls_index
                end_position = cls_index

            if example_index < 5:
                logger.info("*** Example ***")
                logger.info("unique_id: %s" % (unique_id))
                logger.info("example_index: %s" % (example_index))
                logger.info("doc_span_index: %s" % (doc_span_index))
                logger.info("tokens: %s" % " ".join(tokens))
                logger.info("token_to_orig_map: %s" % " ".join([
                    "%d:%d" % (x, y) for (x, y) in token_to_orig_map.items()]))
                logger.info("token_is_max_context: %s" % " ".join([
                    "%d:%s" % (x, y) for (x, y) in token_is_max_context.items()
                ]))
                logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
                logger.info(
                    "input_mask: %s" % " ".join([str(x) for x in input_mask]))
                logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
                if is_training and span_is_impossible:
                    logger.info("impossible example")
                if is_training and not span_is_impossible:
                    answer_text = " ".join(tokens[start_position:(end_position + 1)])
                    logger.info("start_position: %d" % (start_position))
                    logger.info("end_position: %d" % (end_position))
                    logger.info(
                        "answer: %s" % (answer_text))

            features.append(
                InputFeatures(
                    unique_id=unique_id,
                    example_index=example_index,
                    doc_span_index=doc_span_index,
                    tokens=tokens,
                    token_to_orig_map=token_to_orig_map,
                    token_is_max_context=token_is_max_context,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    cls_index=cls_index,
                    p_mask=p_mask,
                    paragraph_len=paragraph_len,
                    start_position=start_position,
                    end_position=end_position,
                    is_impossible=span_is_impossible))
            unique_id += 1

    return features


In [0]:
def load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False):
    """返回 TensorDataset """
    # Load data features from cache or dataset file  加载数据
    input_file = args.dev_file if evaluate else args.train_file
    cached_features_file = os.path.join(os.path.dirname(input_file), 'cached_{}_{}_{}'.format(
        'dev' if evaluate else 'train',
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(args.max_seq_length)
    ))
    # 如果存在缓存文件，且不重新覆盖缓存，并且不需要输出样本，否者，重新读取数据构建样本
    if os.path.exists(cached_features_file) and not args.overwrite_cache and not output_examples:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", input_file)
        examples = read_squad_examples(input_file = input_file,
                                       is_training = not evaluate,
                                       version_2_with_negative = args.version_2_with_negative)
        features = convert_examples_to_features(examples=examples,
                                                tokenizer=tokenizer,
                                                max_seq_length=args.max_seq_length,
                                                doc_stride=args.doc_stride,
                                                max_query_length=args.max_query_length,
                                                is_training=not evaluate)
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
    all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)
    
    if evaluate:
        all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_example_index, all_cls_index, all_p_mask)
    else:
        all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
        all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_start_positions, all_end_positions,
                                all_cls_index, all_p_mask)

    if output_examples:
        return dataset, examples, features
    
    return dataset


### 训练模型

In [0]:
def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    tb_writer = SummaryWriter()
    
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)
    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    # 什么用？用于调整学习率
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps, t_total=t_total)
    
    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.train_batch_size)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
    # set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
        for step, batch in enumerate(epoch_iterator):
            
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'input_ids':       batch[0],
                      'attention_mask':  batch[1],
                      'start_positions': batch[3],
                      'end_positions':   batch[4]}
            if args.model_type != 'distilbert':
                inputs['token_type_ids'] = None if args.model_type == 'xlm' else batch[2]
            if args.model_type in ['xlnet', 'xlm']:
                inputs.update({'cls_index': batch[5],
                               'p_mask':       batch[6]})
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics  是否进行 验证
#                     if args.evaluate_during_training:  # Only evaluate when single GPU otherwise metrics may not average well
#                         results = evaluate(args, model, tokenizer)
#                         for key, value in results.items():
#                             tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                    tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    torch.save(args, os.path.join(output_dir, 'training_args.bin'))
                    logger.info("Saving model checkpoint to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    tb_writer.close()

    return global_step, tr_loss / global_step


主函数

In [0]:
def main(args):
    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
        raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))

    # Setup logging 
    logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)
    
    # 初始化模型
    args.model_type = args.model_type.lower()
    config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path)
    tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path, do_lower_case=args.do_lower_case)
    model = model_class.from_pretrained(args.model_name_or_path, from_tf=bool('.ckpt' in args.model_name_or_path), config=config)

    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        print('加载数据中！')
        train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False)
        print('训练模型！')
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)


    # Save the trained model and the tokenizer
    if args.do_train :
        # Create output directory if needed
        if not os.path.exists(args.output_dir):
            os.makedirs(args.output_dir)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))


In [25]:
main(args)

10/22/2019 03:47:18 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
10/22/2019 03:47:18 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 12

加载数据中！


10/22/2019 03:47:23 - INFO - __main__ -   *** Example ***
10/22/2019 03:47:23 - INFO - __main__ -   unique_id: 1000000000
10/22/2019 03:47:23 - INFO - __main__ -   example_index: 0
10/22/2019 03:47:23 - INFO - __main__ -   doc_span_index: 0
10/22/2019 03:47:23 - INFO - __main__ -   tokens: [CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1 9 6 3 年 被 任 为 主 教 ； 1 9 9 0 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1 9 9 4 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2 0 0 9 年 2 月 离 世 。 范 廷 颂 于 1 9 1 9 年 6 月 1 5 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1 9 4 0 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1 9 4 9 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1 9 5 0 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1 9 5 4 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1 9 6 0 年 因 捍 卫 修 院 的 自 由

训练模型！


10/22/2019 03:55:30 - INFO - transformers.configuration_utils -   Configuration saved in /tmp/debug_squad/checkpoint-50/config.json
10/22/2019 03:55:31 - INFO - transformers.modeling_utils -   Model weights saved in /tmp/debug_squad/checkpoint-50/pytorch_model.bin
10/22/2019 03:55:31 - INFO - __main__ -   Saving model checkpoint to /tmp/debug_squad/checkpoint-50
10/22/2019 03:56:56 - INFO - transformers.configuration_utils -   Configuration saved in /tmp/debug_squad/checkpoint-100/config.json
10/22/2019 03:56:57 - INFO - transformers.modeling_utils -   Model weights saved in /tmp/debug_squad/checkpoint-100/pytorch_model.bin
10/22/2019 03:56:57 - INFO - __main__ -   Saving model checkpoint to /tmp/debug_squad/checkpoint-100
10/22/2019 03:58:22 - INFO - transformers.configuration_utils -   Configuration saved in /tmp/debug_squad/checkpoint-150/config.json
10/22/2019 03:58:23 - INFO - transformers.modeling_utils -   Model weights saved in /tmp/debug_squad/checkpoint-150/pytorch_model.bin


下载文件

In [26]:
!ls

data  runs  sample_data


In [27]:
!tar -zcvf test.tar.gz runs

runs/
runs/Oct22_03-54-04_9b5fcaddf968/
runs/Oct22_03-54-04_9b5fcaddf968/events.out.tfevents.1571716444.9b5fcaddf968.119.0
